# VisualOdometrySLAM

In [119]:
import random

import cv2
import numpy as np
import pandas as pd

def seed_all(seed: int) -> None:
    np.random.seed(seed)
    random.seed(seed)
    cv2.setRNGSeed(seed)

In [120]:
SEED = 42
seed_all(SEED)

In [121]:
from pathlib import Path

DATA_PATH = Path('../data/')
DATA_PATH.mkdir(parents=True, exist_ok=True)

DATA_PATH_VIDEO = DATA_PATH /Path('visual_odometry/')
DATA_PATH_VIDEO.mkdir(parents=True, exist_ok=True)

DATA_PATH_OUTPUT = DATA_PATH / Path('output_data/')
DATA_PATH_OUTPUT.mkdir(parents=True, exist_ok=True)

DATA_PATH_SAVE_MODELS = DATA_PATH / Path('models/')
DATA_PATH_SAVE_MODELS.mkdir(parents=True, exist_ok=True)

DATA_IMGS = Path('../imgs')
DATA_IMGS.mkdir(parents=True, exist_ok=True)

In [122]:
import sys
import os

project_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_path)

## Общие методы

| Параметр                  | Где в YAML                     | Что означает                                  | Как определить / откуда берётся                 |
| ------------------------- | ------------------------------ | --------------------------------------------- | ----------------------------------------------- |
| `fx`, `fy`                | `camera_matrix.data[0]`, `[4]` | Фокусное расстояние по осям X и Y             | Из калибровки (например, `cv2.calibrateCamera`) |
| `cx`, `cy`                | `camera_matrix.data[2]`, `[5]` | Координаты главной точки (оптический центр)   | Центр кадра или из калибровки                   |
| `camera_matrix`           | `camera_matrix.data`           | Внутренняя матрица 3×3                        | Результат калибровки                            |
| `distortion_model`        | `distortion_model`             | Модель дисторсии (`plumb_bob`, `equidistant`) | Выбирается при калибровке, по типу объектива    |
| `distortion_coefficients` | `distortion_coefficients.data` | Коэффициенты искажений (k1, k2, p1, p2, k3)   | Выход `cv2.calibrateCamera`, `Kalibr`, ROS      |
| `image_width` / `height`  | `image_width`, `image_height`  | Размер изображения (в пикселях)               | Известно по сенсору или из примера калибровки   |
| `camera_name`             | `camera_name`                  | Название камеры                               | Задаётся вручную (опционально)                  |


In [123]:
import cv2
import numpy as np
from pathlib import Path
from typing import Union, List


def load_video_frames(video_path: Union[str, Path], target_fps: float) -> List[np.ndarray]:
    """
    Извлекает кадры из видео с заданной частотой и переводит их в оттенки серого.

    :param video_path: путь к видеофайлу
    :param target_fps: целевая частота обработки кадров
    :return: список кадров (в оттенках серого)
    """
    video_path = Path(video_path)
    if not video_path.exists():
        raise FileNotFoundError(f'Видео не найдено по пути: {video_path.resolve()}')

    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        raise IOError(f'Не удалось открыть видеофайл: {video_path}')

    video_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(video_fps / target_fps)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration_sec = total_frames / video_fps
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    print(f"Информация о видео:")
    print(f"- Имя файла: {video_path.name}")
    print(f"- Размер кадра: {frame_width} x {frame_height}")
    print(f"- Оригинальный FPS: {video_fps:.2f}")
    print(f"- Общее число кадров: {total_frames}")
    print(f"- Длительность видео: {duration_sec:.2f} секунд")
    print(f"- Целевая частота обработки: {target_fps} FPS")
    print(f"- Будет обрабатываться каждый {frame_interval}-й кадр")

    processed_frames = []
    frame_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx % frame_interval == 0:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            processed_frames.append(gray)

        frame_idx += 1

    cap.release()
    print(f"Обработано кадров: {len(processed_frames)} при целевом FPS: {target_fps}")
    return processed_frames


def load_kitti_sequence(path: Union[str, Path], grayscale: bool = True) -> List[np.ndarray]:
    image_paths = sorted(Path(path).glob("*.png"))
    frames = []
    for p in image_paths:
        img = cv2.imread(str(p), cv2.IMREAD_GRAYSCALE if grayscale else cv2.IMREAD_COLOR)
        if img is not None:
            frames.append(img)
    return frames


## ORB params

| Параметр        | Тип     | Значение по умолчанию  | Описание и рекомендации                                                                                                         |
| --------------- | ------- | ---------------------- | ------------------------------------------------------------------------------------------------------------------------------- |
| `nfeatures`     | `int`   | 500                    | Максимальное количество фич, которые ORB попытается найти. <br>🔧 Увеличь до 3000–5000 для богатой сцены или сеточной детекции. |
| `scaleFactor`   | `float` | 1.2                    | Масштаб между уровнями пирамиды. <br>Меньше — больше перекрытий, лучше для масштабных сцен, но медленнее.                       |
| `nlevels`       | `int`   | 8                      | Кол-во уровней в пирамиде изображений. <br>🔧 Увеличь до 12–16, если ожидается масштабный диапазон объектов.                    |
| `edgeThreshold` | `int`   | 31                     | Отступ от границ изображения. <br>🔧 Уменьши до 10–15, если хочешь находить фичи у краёв.                                       |
| `firstLevel`    | `int`   | 0                      | Начальный уровень пирамиды (обычно 0). Почти никогда не меняется.                                                               |
| `WTA_K`         | `int`   | 2                      | Метод генерации BRIEF-дескриптора: 2 = быстрее, 3 или 4 = точнее. <br>Используй 2, если нет проблем с точностью.                |
| `scoreType`     | `int`   | `cv2.ORB_HARRIS_SCORE` | Выбор способа ранжирования углов: `cv2.ORB_HARRIS_SCORE` или `cv2.ORB_FAST_SCORE`. <br>`HARRIS` точнее, `FAST` — быстрее.       |
| `patchSize`     | `int`   | 31                     | Размер области вокруг фичи для дескриптора. <br>🔧 31 — стандарт, можно увеличить до 49, если объект крупный.                   |
| `fastThreshold` | `int`   | 20                     | Порог FAST-угла: чем меньше — тем больше фич. <br>🔧 Снизь до 5–10, если ORB пропускает фичи.                                   |


In [124]:
import yaml
from pathlib import Path
from collections import Counter
from typing import Union, List, Optional

import cv2
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt



In [125]:
class MapPoint:
    def __init__(self, coord_3d: np.ndarray, descriptor: np.ndarray):
        self.coord = coord_3d  # (X, Y, Z)
        self.descriptor = descriptor
        self.observations = []  # список (frame_idx, keypoint_idx, (x, y))


class KeyFrame:
    def __init__(self, frame_id: int, image: np.ndarray, keypoints, descriptors, pose: np.ndarray):
        self.id = frame_id
        self.image = image
        self.keypoints = keypoints
        self.descriptors = descriptors
        self.pose = pose        

In [126]:
import math
from collections import Counter
from typing import List, Tuple

import numpy as np
import cv2
import g2o


class MatcherFilterMixin:
    @staticmethod
    def filter_by_ratio_test(knn_matches, kp1, kp2, ratio_thresh):
        good_matches = []
        pts1, pts2 = [], []

        for pair in knn_matches:
            if len(pair) < 2:
                continue
            m, n = pair
            if m.distance < ratio_thresh * n.distance:
                good_matches.append(m)
                pts1.append(kp1[m.queryIdx].pt)
                pts2.append(kp2[m.trainIdx].pt)

        return good_matches, np.float32(pts1), np.float32(pts2)

    @staticmethod
    def filter_by_max_distance(matches, pts1, pts2, max_dist):
        filtered_matches = []
        filtered_pts1, filtered_pts2 = [], []

        for i, m in enumerate(matches):
            if m.distance < max_dist:
                filtered_matches.append(m)
                filtered_pts1.append(pts1[i])
                filtered_pts2.append(pts2[i])

        return filtered_matches, np.float32(filtered_pts1), np.float32(filtered_pts2)

    @staticmethod
    def filter_unique_matches(matches, pts1, pts2):
        seen_trainIdx = set()
        unique_matches = []
        unique_pts1, unique_pts2 = [], []

        for i, m in enumerate(matches):
            if m.trainIdx not in seen_trainIdx:
                seen_trainIdx.add(m.trainIdx)
                unique_matches.append(m)
                unique_pts1.append(pts1[i])
                unique_pts2.append(pts2[i])

        return unique_matches, np.float32(unique_pts1), np.float32(unique_pts2)

    @staticmethod
    def filter_by_ransac_fundamental(matches, pts1, pts2):
        if len(matches) < 8:
            return [], np.array([]), np.array([])

        F, mask = cv2.findFundamentalMat(pts1, pts2, method=cv2.FM_RANSAC, ransacReprojThreshold=1.0)

        inlier_matches = []
        inlier_pts1, inlier_pts2 = [], []

        if mask is not None:
            for i, m in enumerate(matches):
                if mask[i]:
                    inlier_matches.append(m)
                    inlier_pts1.append(pts1[i])
                    inlier_pts2.append(pts2[i])

        return inlier_matches, np.float32(inlier_pts1), np.float32(inlier_pts2)

    @staticmethod
    def filter_by_exclusion_mask(matches, pts1, pts2, mask_exclude_regions: List[Tuple[int, int, int, int]]):
        if not mask_exclude_regions:
            return matches, pts1, pts2

        filtered_matches = []
        filtered_pts1 = []
        filtered_pts2 = []

        for i, m in enumerate(matches):
            x1, y1 = pts1[i]
            x2, y2 = pts2[i]

            excluded = any(
                (xmin <= x1 <= xmax and ymin <= y1 <= ymax) or
                (xmin <= x2 <= xmax and ymin <= y2 <= ymax)
                for (xmin, ymin, xmax, ymax) in mask_exclude_regions
            )

            if not excluded:
                filtered_matches.append(m)
                filtered_pts1.append((x1, y1))
                filtered_pts2.append((x2, y2))

        return filtered_matches, np.float32(filtered_pts1), np.float32(filtered_pts2)

    @staticmethod
    def filter_mutual_matches(
        matcher: cv2.DescriptorMatcher,
        des1: np.ndarray,
        des2: np.ndarray,
        kp1: List[cv2.KeyPoint],
        kp2: List[cv2.KeyPoint],
        matches: List[cv2.DMatch],
        pts1: np.ndarray,
        pts2: np.ndarray
    ) -> Tuple[List[cv2.DMatch], np.ndarray, np.ndarray]:
        """
        Оставляет только взаимные совпадения (A→B и B→A).
        
        :param matcher: объект cv2.DescriptorMatcher (например, BFMatcher или FlannBasedMatcher)
        :param des1: дескрипторы первого изображения
        :param des2: дескрипторы второго изображения
        :param kp1: ключевые точки первого изображения
        :param kp2: ключевые точки второго изображения
        :param matches: прямые совпадения A→B
        :param pts1: координаты точек первого изображения
        :param pts2: координаты точек второго изображения
        :return: (взаимные совпадения, pts1, pts2)
        """
        reverse_matches = matcher.match(des2, des1)
        reverse_set = {(m.trainIdx, m.queryIdx) for m in reverse_matches}

        mutual_matches = []
        mutual_pts1, mutual_pts2 = [], []

        for i, m in enumerate(matches):
            if (m.queryIdx, m.trainIdx) in reverse_set:
                mutual_matches.append(m)
                mutual_pts1.append(pts1[i])
                mutual_pts2.append(pts2[i])

        return mutual_matches, np.float32(mutual_pts1), np.float32(mutual_pts2)

In [127]:
import cv2
import numpy as np
import random

def _to_bgr(img):
    return cv2.cvtColor(img, cv2.COLOR_GRAY2BGR) if len(img.shape) == 2 else img

def _generate_colors(n: int):
    # равномерно распределяем оттенки по кругу HSV и конвертируем в BGR
    hsv = np.zeros((n, 1, 3), dtype=np.uint8)
    hsv[:, 0, 0] = np.linspace(0, 179, n, endpoint=False).astype(np.uint8)   # H
    hsv[:, 0, 1] = 200                                                       # S
    hsv[:, 0, 2] = 255                                                       # V
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)[:, 0, :]
    return [tuple(int(c) for c in bgr[i]) for i in range(n)]

def _draw_matches_colored(img1, kp1, img2, kp2, matches, colors, radius=3, thickness=1):
    img1c, img2c = _to_bgr(img1), _to_bgr(img2)
    h1, w1 = img1c.shape[:2]
    h2, w2 = img2c.shape[:2]
    H = max(h1, h2)
    canvas = np.zeros((H, w1 + w2, 3), dtype=np.uint8)
    canvas[:h1, :w1] = img1c
    canvas[:h2, w1:w1 + w2] = img2c

    for (m, color) in zip(matches, colors):
        p1 = tuple(map(int, kp1[m.queryIdx].pt))
        p2 = tuple(map(int, kp2[m.trainIdx].pt))
        p2_shifted = (int(p2[0] + w1), int(p2[1]))

        cv2.circle(canvas, p1, radius, color, -1, cv2.LINE_AA)
        cv2.circle(canvas, p2_shifted, radius, color, -1, cv2.LINE_AA)
        cv2.line(canvas, p1, p2_shifted, color, thickness, cv2.LINE_AA)

    return canvas

def _sample_matches(matches, sample_ratio: float, random_seed=None):
    if not matches:
        return []
    sample_ratio = max(0.0, min(1.0, float(sample_ratio)))
    n = max(1, int(len(matches) * sample_ratio)) if sample_ratio > 0 else 0
    if random_seed is not None:
        rnd = random.Random(random_seed)
        return rnd.sample(matches, n) if n < len(matches) else matches
    return random.sample(matches, n) if n < len(matches) else matches

In [ ]:
class VisualOdometry(MatcherFilterMixin):
    def __init__(self, 
                 video_path: Union[str, Path],
                 calibration_file: Union[str, Path],                   
                 **params) -> None:
        """
        Инициализация пайплайна визуальной одометрии.

        Загружает калибровку камеры и видео, а также позволяет задать параметры ORB-детектора через `params`.

        :param video_path: путь к видеофайлу (например, .mp4)
        :param calibration_file: путь к YAML-файлу с калибровкой (формат OpenCV / ROS)
        :param target_fps: частота обработки кадров
        :param params: дополнительные параметры, передаваемые в ORB.create(), например:
                       nfeatures=3000, edgeThreshold=10, fastThreshold=5 и т.п.
        """
        self.params: dict = params
        self.video_path = Path(video_path)
        self.calibration_file = Path(calibration_file)
        self.target_fps = self.params.get("target_fps", 2)
        
        self.K: np.ndarray 
        self.distCoeffs: Optional[np.ndarray]
        self.distortion_model: Optional[str]

        self.K, self.distCoeffs, self.distortion_model = self.load_calibration(self.calibration_file)
        self.frames: List[np.ndarray] = self.load_frames(self.video_path)
        
        self.keypoints: List = []  
        self.matches: List = []   
        self.keyframes: List[KeyFrame] = []
        self.mappoints: List[MapPoint] = []
        self.triangulated_points: List[np.ndarray] = []  
        self.poses: List[np.ndarray] = []             

        self.orb = self.create_orb()
        self.matcher = self.create_matcher()
        
    def load_calibration(self, path: Union[str, Path]) -> Tuple[np.ndarray, Optional[np.ndarray], Optional[str]]:
        """
        Универсальная загрузка калибровки:
        - Если .txt → KITTI формат (с префиксом P0: или без)
        - Если .yaml → OpenCV / ROS
        """
        path = Path(path)
        if not path.exists():
            raise FileNotFoundError(f"Файл калибровки не найден: {path.resolve()}")

        if path.suffix == ".txt":
            with path.open("r") as f:
                lines = [line.strip() for line in f if line.strip()]

            # Попробуем сначала с P0:/P1:
            for line in lines:
                if line.startswith("P0:") or line.startswith("P1:"):
                    values = list(map(float, line.split()[1:]))
                    if len(values) == 12:
                        P = np.array(values).reshape(3, 4)
                        return P[:, :3], None, "none"

            # Попробуем первую строку как просто 12 чисел
            for line in lines:
                values = list(map(float, line.split()))
                if len(values) == 12:
                    P = np.array(values).reshape(3, 4)
                    return P[:, :3], None, "none"

            raise ValueError("Неверный формат KITTI-файла. Ожидалась строка из 12 чисел.")

        elif path.suffix in (".yaml", ".yml"):
            import yaml
            with path.open('r') as f:
                data = yaml.safe_load(f)

            K_data = data["camera_matrix"]["data"]
            K = np.array(K_data, dtype=np.float64).reshape(3, 3)

            dist_data = data.get("distortion_coefficients", {}).get("data", [])
            distCoeffs = np.array(dist_data, dtype=np.float64) if dist_data else None

            model = data.get("distortion_model", None)
            return K, distCoeffs, model

        else:
            raise ValueError(f"Неизвестный формат файла калибровки: {path}") 

    def load_frames(self, path: Union[str, Path]) -> List[np.ndarray]:
        """
        Загружает последовательность кадров:
        - если путь — директория, то PNG (KITTI)
        - если файл — то видео с заданной частотой
        Все кадры приводятся к оттенкам серого.
        """
        path = Path(path)
        frames = []

        if path.is_dir():
            # KITTI-подобная последовательность
            image_paths = sorted(path.glob("*.png"))
            if not image_paths:
                raise FileNotFoundError(f"Не найдено изображений в директории: {path.resolve()}")

            for p in image_paths:
                img = cv2.imread(str(p), cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    frames.append(img)

            print(f"[KITTI] Загружено {len(frames)} кадров из {path.name}")

        else:
            # Видеофайл
            if not path.exists():
                raise FileNotFoundError(f"Видео не найдено: {path.resolve()}")

            cap = cv2.VideoCapture(str(path))
            if not cap.isOpened():
                raise IOError(f"Не удалось открыть видео: {path}")

            video_fps = cap.get(cv2.CAP_PROP_FPS)
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            frame_interval = max(1, int(video_fps / self.target_fps)) if video_fps else 1

            print(f"[Видео] Имя: {path.name}")
            print(f"- FPS: {video_fps:.2f}, кадры: {total_frames}, интервал: {frame_interval}")

            frame_idx = 0
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                if frame_idx % frame_interval == 0:
                    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                    frames.append(gray)
                frame_idx += 1

            cap.release()
            print(f"[Видео] Загружено {len(frames)} кадров с шагом {frame_interval}")

        if not frames:
            raise RuntimeError("Не удалось загрузить ни одного кадра.")
        return frames

    def clear_data(self):
        """
        Очищает все внутренние списки и данные.
        Используется для перезапуска пайплайна.
        """
        self.keypoints.clear()
        self.matches.clear()
        self.keyframes.clear()
        self.mappoints.clear()
        self.poses.clear()
        self.triangulated_points.clear()

    # def create_matcher(self) -> cv2.DescriptorMatcher:
    #     FLANN_INDEX_LSH = 6
    #     index_params = dict(
    #         algorithm=FLANN_INDEX_LSH,
    #         table_number=self.params.get("flann_table_number", 6),
    #         key_size=self.params.get("flann_key_size", 12), 
    #         multi_probe_level=self.params.get("flann_multi_probe_level", 1)
    #     )
    #     search_params = dict(
    #         checks=self.params.get("flann_checks", 50)
    #     )
    #     return cv2.FlannBasedMatcher(index_params, search_params)
    
    def create_matcher(self):
        return cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
    
    def create_orb(self) -> cv2.ORB:
        """
        Создаёт ORB-детектор с параметрами из конструктора.
        :return: объект cv2.ORB
        """
        orb_params = {
            "nfeatures": self.params.get("nfeatures", 1000),
            "scaleFactor": self.params.get("scaleFactor", 1.2),
            "nlevels": self.params.get("nlevels", 8),
            "edgeThreshold": self.params.get("edgeThreshold", 31),
            "fastThreshold": self.params.get("fastThreshold", 20)
        }
        return cv2.ORB.create(**orb_params)

    def anms_ssc(self, keypoints, num_ret_points, tolerance, cols, rows):
        """
        SSC (Suppression via Square Covering, aka fast ANMS).
        Отбирает равномерно распределённые ключевые точки по всему изображению.
        Возвращает индексы выбранных точек (а не сами точки!).
        """
        # print(f"[ANMS_SSC] Входных keypoints: {len(keypoints)}, желаемое число: {num_ret_points}")
        if len(keypoints) <= num_ret_points:
            # print(f"[ANMS_SSC] Keypoints меньше или равно num_ret_points, возвращаем все.")
            return list(range(len(keypoints)))

        exp1 = rows + cols + 2 * num_ret_points
        exp2 = (
            4 * cols
            + 4 * num_ret_points
            + 4 * rows * num_ret_points
            + rows * rows
            + cols * cols
            - 2 * rows * cols
            + 4 * rows * cols * num_ret_points
        )
        exp3 = math.sqrt(exp2)
        exp4 = num_ret_points - 1

        sol1 = -round(float(exp1 + exp3) / exp4)
        sol2 = -round(float(exp1 - exp3) / exp4)
        high = sol1 if (sol1 > sol2) else sol2
        low = math.floor(math.sqrt(len(keypoints) / num_ret_points))

        # print(f"[ANMS_SSC] Инициализация: low={low}, high={high}")

        prev_width = -1
        result_list = []
        complete = False
        k = num_ret_points
        k_min = round(k - (k * tolerance))
        k_max = round(k + (k * tolerance))

        iters = 0
        while not complete:
            width = low + (high - low) / 2
            if width == prev_width or low > high:
                # print(f"[ANMS_SSC] Завершение поиска. Предыдущая ширина: {prev_width}, текущая: {width}, low: {low}, high: {high}")
                break  # выход из цикла если больше нечего делить

            c = width / 2  # размер ячейки в grid
            num_cell_cols = int(math.floor(cols / c))
            num_cell_rows = int(math.floor(rows / c))
            covered_vec = [
                [False for _ in range(num_cell_cols + 1)] for _ in range(num_cell_rows + 1)
            ]
            result = []

            for i, kp in enumerate(keypoints):
                row = int(math.floor(kp.pt[1] / c))
                col = int(math.floor(kp.pt[0] / c))
                if not covered_vec[row][col]:
                    result.append(i)
                    # Закрываем область квадратом с длиной width вокруг точки
                    row_min = max(0, int(row - math.floor(width / c)))
                    row_max = min(num_cell_rows, int(row + math.floor(width / c)))
                    col_min = max(0, int(col - math.floor(width / c)))
                    col_max = min(num_cell_cols, int(col + math.floor(width / c)))
                    for row_to_cover in range(row_min, row_max + 1):
                        for col_to_cover in range(col_min, col_max + 1):
                            covered_vec[row_to_cover][col_to_cover] = True

            # print(f"[ANMS_SSC] Итерация {iters}: width={width:.2f}, выбранных точек={len(result)}")
            iters += 1

            if k_min <= len(result) <= k_max:
                # print(f"[ANMS_SSC] Найдено решение: {len(result)} точек (цель {k}±{int(k * tolerance)})")
                result_list = result
                complete = True
            elif len(result) < k_min:
                high = width - 1
            else:
                low = width + 1
            prev_width = width

        # Если не нашли ровно нужное число, берём то, что получилось
        if not result_list:
            # print(f"[ANMS_SSC] Используем последнее найденное решение: {len(result)} точек.")
            result_list = result

        # print(f"[ANMS_SSC] Финальное число выбранных точек: {len(result_list)}")
        return result_list

    def detect_and_compute(self, image: np.ndarray) -> Tuple[List[cv2.KeyPoint], np.ndarray]:
        use_grid = self.params.get("use_grid", False)
        use_anms = self.params.get("use_anms", False)
        anms_count = self.params.get("anms_count", 10)
        anms_tolerance = self.params.get("anms_tolerance", 0.1)

        if not use_grid:            
            fast = cv2.FastFeatureDetector_create(threshold=10, nonmaxSuppression=True)
            kp_all = fast.detect(image, None)
            kp_all, des_all = self.orb.compute(image, kp_all)
            # kp_all, des_all = self.orb.detectAndCompute(image, None)
        else:
            num_rows = self.params.get("grid_rows", 8)
            num_cols = self.params.get("grid_cols", 8)
            h, w = image.shape
            kp_all, des_all = [], []
            for i in range(num_rows):
                for j in range(num_cols):
                    x0 = int(j * w / num_cols)
                    x1 = int((j + 1) * w / num_cols)
                    y0 = int(i * h / num_rows)
                    y1 = int((i + 1) * h / num_rows)
                    roi = image[y0:y1, x0:x1]
                    kp, des = self.orb.detectAndCompute(roi, None)
                    for k in kp:
                        k.pt = (k.pt[0] + x0, k.pt[1] + y0)
                    if kp:
                        kp_all.extend(kp)
                        if des is not None:
                            des_all.append(des)
            if des_all:
                des_all = np.vstack(des_all)
            else:
                des_all = None

        print(f"[Debug] Найдено {len(kp_all)} ключевых точек")
        
        if use_anms and kp_all and des_all is not None:
            kp_sorted = sorted(kp_all, key=lambda kp: kp.response, reverse=True)
            idx_map = {id(kp): i for i, kp in enumerate(kp_all)}
            kp_sorted_indices = [idx_map[id(kp)] for kp in kp_sorted]

            h, w = image.shape[:2]
            idx_ssc = self.anms_ssc(kp_sorted, anms_count, anms_tolerance, w, h)
            kp_anms = [kp_sorted[i] for i in idx_ssc]
            idx_final = [kp_sorted_indices[i] for i in idx_ssc]
            des_anms = des_all[idx_final]

            kp_all, des_all = kp_anms, des_anms
            print(f"[Debug] После ANMS осталось {len(kp_all)} точек")

        return kp_all, des_all
    
    def add_keyframe(self, frame_id: int, image: np.ndarray, keypoints, descriptors, pose: np.ndarray):
        kf = KeyFrame(frame_id, image, keypoints, descriptors, pose)
        self.keyframes.append(kf)

    def add_mappoint(self, coord_3d: np.ndarray, frame_id: int, kp_idx: int, pt2d: Tuple[float, float], descriptor: np.ndarray):
        mp = MapPoint(coord_3d, descriptor)
        mp.observations.append((frame_id, kp_idx, pt2d))
        self.mappoints.append(mp)
           
    def get_matches(
        self,
        kp1: List[cv2.KeyPoint],
        des1: np.ndarray,
        kp2: List[cv2.KeyPoint],
        des2: np.ndarray
    ) -> Tuple[List[cv2.KeyPoint], List[cv2.KeyPoint], List[cv2.DMatch], np.ndarray, np.ndarray]:

        if des1 is None or des2 is None or len(kp1) == 0 or len(kp2) == 0:
            return [], [], [], np.array([]), np.array([])

        # 1. Первичное сопоставление дескрипторов
        knn_matches = self.matcher.knnMatch(des1, des2, k=2)

        # 2. Lowe's ratio test
        ratio_thresh = self.params.get("ratio_thresh", 0.6)
        good_matches, pts1, pts2 = self.filter_by_ratio_test(knn_matches, kp1, kp2, ratio_thresh)

        # 3. Ограничение по дистанции
        max_dist = self.params.get("max_match_distance", 60)
        good_matches, pts1, pts2 = self.filter_by_max_distance(good_matches, pts1, pts2, max_dist)

        # 4. Удаление повторяющихся trainIdx
        good_matches, pts1, pts2 = self.filter_unique_matches(good_matches, pts1, pts2)

        # 5. Геометрическая фильтрация через фундаментальную матрицу
        good_matches, pts1, pts2 = self.filter_by_ransac_fundamental(good_matches, pts1, pts2)

        # # 6. Исключение по зонам маски интереса (если указаны)
        # mask_exclude_regions = self.params.get("mask_exclude_regions", [])
        # good_matches, pts1, pts2 = self.filter_by_exclusion_mask(good_matches, pts1, pts2, mask_exclude_regions)

        # 7. Взаимные совпадения A→B и B→A
        if self.params.get("use_mutual_check", False):
            good_matches, pts1, pts2 = self.filter_mutual_matches(
                self.matcher, des1, des2, kp1, kp2, good_matches, pts1, pts2
            )
        
        self.keypoints.append((kp1, kp2))
        self.matches.append(good_matches)

        return kp1, kp2, good_matches, pts1, pts2
    
    def estimate_pose_opencv(self, pts1: np.ndarray, pts2: np.ndarray) -> Optional[Tuple[np.ndarray, np.ndarray, np.ndarray]]:
        """
        Оценивает относительную позу между двумя изображениями с использованием Essential-матрицы.
        
        :param pts1: сопоставленные точки из предыдущего кадра (Nx2)
        :param pts2: сопоставленные точки из текущего кадра (Nx2)
        :return: (T, R, t) — 4×4 матрица трансформации, вращение и трансляция
        """
        if pts1.shape[0] < 8 or pts2.shape[0] < 8:
            print("Недостаточно точек для оценки движения.")
            return None

        E, mask = cv2.findEssentialMat(
            pts1, pts2, self.K,
            method=cv2.RANSAC,
            prob=0.999,
            threshold=1.0
        )

        if E is None or mask is None or mask.sum() < 8:
            print("Не удалось вычислить матрицу Essential.")
            return None

        _, R, t, _ = cv2.recoverPose(E, pts1, pts2, self.K, mask=mask)

        t = t.ravel()
        t = t / np.linalg.norm(t)

        T = self._form_transformation_matrix(R, t)
        return T, R, t
    
    @staticmethod
    def _form_transformation_matrix(R: np.ndarray, t: np.ndarray) -> np.ndarray:
        """
        Формирует 4×4 однородную матрицу трансформации из R и t.
        """
        T = np.eye(4, dtype=np.float64)
        T[:3, :3] = R
        T[:3, 3] = t.ravel()
        return T

    def decompose_essential_custom(self, E: np.ndarray, pts1: np.ndarray, pts2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """
        Декомпозирует матрицу Essential и выбирает правильную пару (R, t)
        на основе cheirality check (только если Z > 0 в обеих камерах).
        Также масштабирует t.
        """

        def triangulate_and_score(R, t):
            P0 = self.K @ np.hstack((np.eye(3), np.zeros((3, 1))))
            P1 = self.K @ np.hstack((R, t.reshape(3, 1)))

            points_4d = cv2.triangulatePoints(P0, P1, pts1.T, pts2.T)
            points_3d = points_4d[:3, :] / points_4d[3, :]

            points_3d_cam2 = R @ points_3d + t.reshape(3, 1)

            z1 = points_3d[2, :] > 0
            z2 = points_3d_cam2[2, :] > 0
            valid = z1 & z2
            num_valid = np.sum(valid)

            if num_valid >= 2:
                d1 = np.linalg.norm(np.diff(points_3d.T[valid], axis=0), axis=1)
                d2 = np.linalg.norm(np.diff(points_3d_cam2.T[valid], axis=0), axis=1)
                scale = np.mean(d1 / d2) if np.all(d2 > 1e-6) else 1.0
            else:
                scale = 1.0

            return num_valid, scale

        R1, R2, t = cv2.decomposeEssentialMat(E)
        t = t.squeeze()
        candidates = [(R1,  t), (R1, -t), (R2,  t), (R2, -t)]

        best_idx = -1
        best_score = -1
        best_scale = 1.0

        for i, (R_cand, t_cand) in enumerate(candidates):
            score, scale = triangulate_and_score(R_cand, t_cand)
            if score > best_score:
                best_score = score
                best_idx = i
                best_scale = scale

        R_best, t_best = candidates[best_idx]
        t_best = t_best / np.linalg.norm(t_best) * best_scale
        return R_best, t_best
    
    def estimate_pose_custom(self, pts1: np.ndarray, pts2: np.ndarray) -> Optional[Tuple[np.ndarray, np.ndarray, np.ndarray]]:
        """
        Оценивает трансформацию (R, t) между двумя изображениями:
        - извлекает матрицу Essential;
        - вручную декомпозирует;
        - выбирает правильную пару (R, t) через cheirality check;
        - формирует 4×4 матрицу T.
        """
        if pts1.shape[0] < 8 or pts2.shape[0] < 8:
            print("Недостаточно точек для вычисления матрицы Essential.")
            return None

        E, mask = cv2.findEssentialMat(pts1, pts2, self.K, method=cv2.RANSAC, threshold=1.0, prob=0.999)
        if E is None or mask is None or mask.sum() < 8:
            print("Не удалось найти матрицу Essential.")
            return None

        R, t = self.decompose_essential_custom(E, pts1, pts2)
        T = self._form_transformation_matrix(R, t)
        return T, R, t

    @staticmethod
    def filter_pointcloud(points: np.ndarray,
                        z_min: float = 0.1,
                        z_max: float = 10.0,
                        x_range: Optional[Tuple[float, float]] = None,
                        y_range: Optional[Tuple[float, float]] = None,
                        voxel_size: Optional[float] = 0.05,
                        apply_statistical: bool = True,
                        nb_neighbors: int = 20,
                        std_ratio: float = 2.0) -> np.ndarray:
        """
        Фильтрует облако точек на основе геометрических и статистических критериев.

        :param points: Массив 3D-точек формы (N, 3)
        
        :param z_min: Минимально допустимое значение глубины Z. Удаляются точки ближе, чем z_min.
        :param z_max: Максимально допустимая глубина Z. Удаляются слишком дальние точки.
        
        :param x_range: Кортеж (min_x, max_x), ограничивающий координаты X. Если None — не фильтруется.
        :param y_range: Кортеж (min_y, max_y), ограничивающий координаты Y. Если None — не фильтруется.
        
        :param voxel_size: Размер ячейки в voxel grid фильтрации. Если None — voxel-фильтрация не применяется.
                        Используется для упрощения и разреживания точек (например, 0.1 = 5 см).
        
        :param nb_neighbors: Число соседей для оценки плотности (используется при статистической фильтрации).
        :param std_ratio: Порог удаления выбросов — точки, чья средняя дистанция отклоняется на std_ratio,
                        будут удалены.
        
        :return: Отфильтрованное облако точек (M, 3)
        """
        # Фильтрация по глубине и NaN/Inf
        mask = (points[:, 2] > z_min) & (points[:, 2] < z_max)
        mask &= np.isfinite(points).all(axis=1)

        # Фильтрация по X, Y (если заданы)
        if x_range:
            mask &= (points[:, 0] > x_range[0]) & (points[:, 0] < x_range[1])
        if y_range:
            mask &= (points[:, 1] > y_range[0]) & (points[:, 1] < y_range[1])

        # Применение маски
        points = points[mask]
        if len(points) == 0:
            return points

        # Конвертация в формат Open3D
        import open3d as o3d
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(points)

        # Voxel Grid — упрощение
        if voxel_size:
            pcd = pcd.voxel_down_sample(voxel_size=voxel_size)

        # Статистическая фильтрация выбросов
        if apply_statistical:
            pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=nb_neighbors, std_ratio=std_ratio)

        return np.asarray(pcd.points)
    
    def triangulate_points(
        self,
        pts1: np.ndarray,
        pts2: np.ndarray,
        T1: np.ndarray,
        T2: np.ndarray,
        z_min: float = 0.01,
        z_max: float = 200.0,
    ) -> Optional[Tuple[np.ndarray, np.ndarray]]:
        """
        Триангулирует 3D-точки и возвращает (points_3d, keep_idx),
        где keep_idx — индексы исходных соответствий pts1/pts2/matches,
        прошедших маску (finite, z в диапазоне).
        """
        if pts1.shape[0] < 8 or pts2.shape[0] < 8:
            return None

        P1 = self.K @ T1[:3, :]
        P2 = self.K @ T2[:3, :]

        points_4d = cv2.triangulatePoints(P1, P2, pts1.T, pts2.T)   # 4×N
        points_3d = (points_4d[:3, :] / points_4d[3, :]).T          # N×3

        finite = np.isfinite(points_3d).all(axis=1)
        z = points_3d[:, 2]
        mask = finite & (z > z_min) & (z < z_max)

        keep_idx = np.where(mask)[0]
        kept_points = points_3d[keep_idx]

        print(f"[Triangulate] Всего: {len(points_3d)} | валидных Z∈({z_min},{z_max}): {len(kept_points)}")

        if len(kept_points) == 0:
            return None

        return kept_points, keep_idx
    
    def get_first_valid_frame(self) -> Tuple[int, np.ndarray, List[cv2.KeyPoint], np.ndarray]:
        for i, img in enumerate(self.frames):
            kp, des = self.detect_and_compute(img)
            if des is not None and len(kp) > 0:
                return i, img, kp, des
        raise RuntimeError("Не найдено ни одного кадра с валидными фичами.")
    
    def scale_translation_by_triangulation(self, R, t, pts1, pts2):
        """
        Масштабирует вектор t, используя триангуляцию точек.
        """
        T_temp = np.eye(4)
        T_temp[:3, :3] = R
        T_temp[:3, 3] = t.ravel()

        X_world = self.triangulate_points(pts1, pts2, np.eye(4), T_temp)

        if X_world is None or len(X_world) < 10:
            print("[Scale] Недостаточно точек для оценки масштаба.")
            return t

        depths = np.linalg.norm(X_world, axis=1)
        median_depth = np.median(depths)

        print(f"[Scale] Кол-во точек для оценки: {len(X_world)}")
        print(f"[Scale] Глубина точек: mean={depths.mean():.3f}, median={median_depth:.3f}, min={depths.min():.3f}, max={depths.max():.3f}")
        print(f"[Scale] Норма исходного t: {np.linalg.norm(t):.3f}")

        if median_depth > 1e-6:
            scale_factor = 1.0 / median_depth
            t_scaled = t * scale_factor
            print(f"[Scale] Масштаб применён: factor={scale_factor:.3f}, норма нового t={np.linalg.norm(t_scaled):.3f}")
        else:
            t_scaled = t
            print("[Scale] Ошибка: median_depth слишком мала, масштаб не изменён")

        return t_scaled

    def _log_mappoint_debug(self, mp, name="MapPoint"):
        print(f"[DEBUG:{name}] 3D координаты: {mp.coord}")
        print(f"[DEBUG:{name}] Кол-во наблюдений: {len(mp.observations)}")
        for (fid, kp_idx, pt2d) in mp.observations:
            print(f"[DEBUG:{name}]   Наблюдение → Frame #{fid}, KeyPoint #{kp_idx}, 2D: {pt2d}")

    def bundle_adjustment_g2o(self, last_frame_size: int = 5):
        """
        BA в стиле ORB-SLAM:
        - наблюдения переводим в нормализованные undistorted координаты (x,y),
        т.е. в систему pinhole с f=1, cx=cy=0;
        - в g2o задаём камеру CameraParameters(1, (0,0), 0);
        - робастное ядро и разумные веса.
        """
        ba_obs_size = self.params.get("ba_obs_size", 2)

        optimizer = g2o.SparseOptimizer()
        solver = g2o.BlockSolverX(g2o.LinearSolverDenseX())
        optimizer.set_algorithm(g2o.OptimizationAlgorithmLevenberg(solver))
        optimizer.set_verbose(True)

        # --- окно по keyframes ---
        keyframes = self.keyframes[-last_frame_size:]
        if len(keyframes) < 2:
            print("[g2o] Недостаточно KeyFrame для BA.")
            return
        pose_id = {kf.id: idx for idx, kf in enumerate(keyframes)}

        # --- отбор валидных точек ---
        total_before = len(self.mappoints)
        mappoints = [mp for mp in self.mappoints
                    if len(mp.observations) >= ba_obs_size and np.isfinite(mp.coord).all()]
        print(f"[g2o] MapPoints до фильтрации по наблюдениям: {total_before}")
        print(f"[g2o] MapPoints после фильтрации: {len(mappoints)}")
        print(f"[g2o] Отфильтровано: {total_before - len(mappoints)}")

        offset = len(keyframes)

        # --- вершины камер ---
        for idx, kf in enumerate(keyframes):
            v = g2o.VertexSE3Expmap()
            v.set_id(idx)
            se3 = g2o.SE3Quat(kf.pose[:3, :3], kf.pose[:3, 3])
            v.set_estimate(se3)
            if idx == 0:  # фиксируем только первую камеру для устранения gauge
                v.set_fixed(True)
            optimizer.add_vertex(v)

        # --- вершины точек ---
        for i, mp in enumerate(mappoints):
            v = g2o.VertexPointXYZ()
            v.set_id(offset + i)
            v.set_estimate(mp.coord)
            v.set_marginalized(True)
            optimizer.add_vertex(v)

        # --- параметры камеры: f=1, cx=cy=0 (измерения уже нормализованы) ---
        cam = g2o.CameraParameters(1.0, np.array([0.0, 0.0]), 0.0)
        cam.set_id(0)
        optimizer.add_parameter(cam)

        # --- хелперы нормализации/undistort ---
        fx, fy = float(self.K[0, 0]), float(self.K[1, 1])
        cx, cy = float(self.K[0, 2]), float(self.K[1, 2])

        def norm_from_pixels(u, v):
            # если нет дисторсии (или model == "none") — просто нормализуем по K
            x = (u - cx) / fx
            y = (v - cy) / fy
            return np.array([x, y], dtype=np.float64)

        def undistort_and_norm(u, v):
            # cv2.undistortPoints без P возвращает нормализованные координаты
            pts = np.array([[[u, v]]], dtype=np.float64)
            und = cv2.undistortPoints(pts, self.K, self.distCoeffs)  # -> (1,1,2) нормализованные (x,y)
            return und.reshape(2).astype(np.float64)

        use_undistort = (self.distCoeffs is not None) and (self.distortion_model is None or self.distortion_model != "none")

        # Веса: σ ~ 1.5px в нормализованных коорд → σ_norm ≈ 1.5 / f̄
        f_mean = 0.5 * (fx + fy) if (fx > 0 and fy > 0) else 1.0
        sigma_norm = 1.5 / f_mean
        info = np.identity(2) * (1.0 / (sigma_norm * sigma_norm))

        # --- рёбра наблюдений ---
        edges = 0
        for i, mp in enumerate(mappoints):
            pid = offset + i
            for (fid, _, (u, v)) in mp.observations:
                if fid not in pose_id:
                    continue
                cam_idx = pose_id[fid]

                meas = undistort_and_norm(u, v) if use_undistort else norm_from_pixels(u, v)

                e = g2o.EdgeProjectXYZ2UV()
                e.set_vertex(0, optimizer.vertex(pid))       # точка
                e.set_vertex(1, optimizer.vertex(cam_idx))   # камера
                e.set_measurement(meas)
                e.set_information(info)

                rk = g2o.RobustKernelHuber()
                rk.set_delta(1.0)  # в нормализованных координатах
                e.set_robust_kernel(rk)

                e.set_parameter_id(0, 0)  # CameraParameters(1, (0,0))
                optimizer.add_edge(e)
                edges += 1

        print(f"[g2o] Оптимизация: {len(keyframes)} камер, {len(mappoints)} точек, {edges} наблюдений")

        optimizer.initialize_optimization()
        iters = int(self.params.get("g2o_optimize_step", 40))
        optimizer.optimize(iters)

        # --- обновление поз ---
        print("[g2o] --- Изменения поз камер ---")
        for idx, kf in enumerate(keyframes):
            old_T = kf.pose.copy()
            T_new = optimizer.vertex(idx).estimate().matrix()
            kf.pose = T_new
            self.poses[kf.id] = T_new

            delta_t = np.linalg.norm(T_new[:3, 3] - old_T[:3, 3])
            R_diff = T_new[:3, :3] @ old_T[:3, :3].T
            angle = np.arccos(np.clip((np.trace(R_diff) - 1) * 0.5, -1.0, 1.0)) * 180.0 / np.pi
            print(f"[g2o] KeyFrame {kf.id}: ∆t = {delta_t:.4f} m, ∆R = {angle:.2f}°")

        # --- обновление точек ---
        print("[g2o] --- Изменения MapPoints ---")
        shifts = []
        for i, mp in enumerate(mappoints):
            old_p = mp.coord.copy()
            new_p = np.asarray(optimizer.vertex(offset + i).estimate()).ravel()
            if np.all(np.isfinite(new_p)):
                mp.coord = new_p
            shifts.append(np.linalg.norm(mp.coord - old_p))

        if shifts:
            shifted = [s for s in shifts if s > 1e-4]
            print(f"[g2o] Среднее смещение:  {np.mean(shifts):.4f} м")
            print(f"[g2o] Максимальное:      {np.max(shifts):.4f} м")
            print(f"[g2o] Минимальное:       {np.min(shifts):.4f} м")
            print(f"[g2o] Смещённых точек:   {len(shifted)} из {len(shifts)}")
        else:
            print("[g2o] Нет изменений точек")

        print("[g2o] Bundle Adjustment завершён.")

        
    def filter_mappoints_depth(
        self,
        z_min: float = 0.1,
        z_max: float = 100.0,
        x_range: tuple = None,
        y_range: tuple = None
    ):
        """
        Удаляет некорректные или слишком близкие/далёкие MapPoints после инициализации.
        """

        before = len(self.mappoints)
        filtered = []

        for mp in self.mappoints:
            coord = mp.coord

            if not np.isfinite(coord).all():
                continue

            if coord[2] < z_min or coord[2] > z_max:
                continue

            if x_range and not (x_range[0] <= coord[0] <= x_range[1]):
                continue
            if y_range and not (y_range[0] <= coord[1] <= y_range[1]):
                continue

            filtered.append(mp)

        self.mappoints = filtered
        after = len(self.mappoints)

        print(f"[Filter] MapPoints до фильтрации: {before}, после: {after}")
        print(f"[Filter] Удалено {before - after} точек")
        
    def filter_mappoints_by_observations(self, min_obs=2):
        before = len(self.mappoints)
        self.mappoints = [mp for mp in self.mappoints if len(mp.observations) >= min_obs]
        after = len(self.mappoints)
        print(f"[Filter] MapPoints по наблюдениям: до = {before}, после = {after}")

    def print_mappoint_observation_stats(self, max_n=6):
        counter = Counter()
        for mp in self.mappoints:
            n = len(mp.observations)
            if n <= max_n:
                counter[n] += 1
            else:
                counter[f">{max_n}"] += 1

        print("\n[MapPoints] Распределение по числу наблюдений:")
        print("| Число наблюдений | Кол-во точек |")
        print("|------------------|---------------|")
        for i in range(1, max_n + 1):
            print(f"| {i:<17} | {counter.get(i, 0):<13} |")
        if f">{max_n}" in counter:
            print(f"| >{max_n:<17} | {counter[f'>{max_n}']:<13} |")

    def init_map(self):
        print("[Init] Запуск инициализации карты...")

        start_idx, img_ref, kp_ref, des_ref = self.get_first_valid_frame()
        print(f"[Init] Первый кадр: #{start_idx}, найдено {len(kp_ref)} ключевых точек.")

        T_prev = np.eye(4)
        self.add_keyframe(start_idx, img_ref, kp_ref, des_ref, T_prev)
        self.poses.append(T_prev)

        num_kfs_target = self.params.get("init_keyframes", 5)
        current_idx = start_idx
                
        for i in range(start_idx + 1, len(self.frames)):
            print(f"[Init:{i}] Обработка кадра #{i}..............")
            
            img_curr = self.frames[i]
            kp_curr, des_curr = self.detect_and_compute(img_curr)

            if des_curr is None or len(kp_curr) == 0:
                print(f"[Init:{i}] Нет валидных дескрипторов, пропускаем кадр.")
                continue

            kp1, kp2, matches, pts1, pts2 = self.get_matches(kp_ref, des_ref, kp_curr, des_curr)
            print(f"[Init:{i}] Сопоставлено {len(matches)} фич между кадрами {current_idx} и {i}")
            if len(matches) < self.params.get("min_matches", 50):
                print(f"[Init:{i}] Недостаточно хороших матчей: {len(matches)}, пробуем следующий кадр.")
                continue
            
            # self.visualize_matches_once(
            #     self.frames[current_idx],
            #     self.frames[i],
            #     kp1,
            #     kp2,
            #     matches,
            #     window_name=f"Matches {current_idx}->{i}",
            #     fraction=1.0
            # )
            
            mean_parallax = np.mean(np.linalg.norm(pts1 - pts2, axis=1))   
            print(f"[Init:{i}] Средний параллакс: {mean_parallax:.2f}")         
            if mean_parallax < self.params.get("min_parallax", 5.0):
                print(f"[Init:{i}] Малый параллакс ({mean_parallax:.2f}) – пробуем следующий кадр.")
                continue

            T_rel, R, t = self.estimate_pose_opencv(pts2, pts1)
            # T_rel, R, t = self.estimate_pose_custom(pts1, pts2)
            
            # if T_rel[2, 3] < 0:
            #     print("[FixDirection] T_rel указывает назад. Инвертируем.")
            #     T_rel = np.linalg.inv(T_rel)
            #     R = T_rel[:3, :3]
            #     t = T_rel[:3, 3]
            
            if T_rel is None:
                print(f"[Init:{i}] Не удалось оценить позу, пробуем следующий кадр.")
                continue   
            
            T_prev = self.poses[-1]
            T_curr = T_prev @ T_rel
            self.poses.append(T_curr)
            t_global = T_curr[:3, 3]
            print(f"[Pose:{i}] Камера на позиции: X={t_global[0]:.3f}, Y={t_global[1]:.3f}, Z={t_global[2]:.3f}")

            self.add_keyframe(i, img_curr, kp_curr, des_curr, T_curr)
            print(f"[Init:{i}] Добавлен KeyFrame #{i}")

            X_world, keep_idx = self.triangulate_points(pts1, pts2, T_curr, T_prev)
            # X_world, keep_idx = self.triangulate_points(pts1, pts2, T_prev, T_curr)
            print(f"[Init:{i}] Триангуляция дала {len(X_world)} 3D-точек.")
            if X_world is None or len(X_world) < 30:
                print(f"[Init:{i}] Триангуляция не дала достаточно точек.")
                continue

            added = 0
            added_obs = 0

            for k, p in zip(keep_idx, X_world):
                m = matches[k]
                kp_idx_ref  = m.queryIdx
                kp_idx_curr = m.trainIdx

                desc       = des_curr[kp_idx_curr]
                pt2d_ref   = tuple(pts1[k])
                pt2d_curr  = tuple(pts2[k])

                mp = self.find_mappoint_for_keypoint(current_idx, kp_idx_ref)

                if mp is None:
                    self.add_mappoint(
                        coord_3d=p,
                        frame_id=i,
                        kp_idx=kp_idx_curr,
                        pt2d=pt2d_curr,
                        descriptor=desc
                    )
                    added += 1
                else:
                    if all(obs[0] != i or obs[1] != kp_idx_curr for obs in mp.observations):
                        mp.observations.append((i, kp_idx_curr, pt2d_curr))
                        added_obs += 1

                        # if len(mp.observations) >= 3:
                        #     mp_index = self.mappoints.index(mp)
                        #     self.show_mappoint_observations(mp_index, text=f"After adding Frame {i}")

            print(f"[Init:{i}] Добавлено {added} новых 3D-точек после фильрации.")
            print(f"[Init:{i}] Добавлено {added_obs} новых наблюдений к существующим MapPoints.")
            print(f"[Init:{i}] KeyFrames: {len(self.keyframes)} | MapPoints: {len(self.mappoints)}")

            kp_ref, des_ref = kp_curr, des_curr
            current_idx = i
            
            if len(self.keyframes) >= num_kfs_target:
                print(f"[Init:{i}] Достигнуто {num_kfs_target} KeyFrame. Инициализация завершена.")
                break
            
            # self.visualize_map_and_camera(self.poses[-1], desc=f"KeyFrame #{i}", min_observations=1)
            
        last_frame_size = self.params.get("last_frame_size", 5)
        self.bundle_adjustment_g2o(last_frame_size=last_frame_size)
                
        self.filter_mappoints_depth(
            z_min=self.params.get("filter_z_min", 0.1),
            z_max=self.params.get("filter_z_max", 100.0),
            x_range=self.params.get("filter_x_range", (-100, 100)),
            y_range=self.params.get("filter_y_range", (-100, 100)),
        )
        self.print_mappoint_observation_stats(max_n=6)
        self.filter_mappoints_by_observations(min_obs=2)
        
        print(f"[Init]  KeyFrames: {len(self.keyframes)}")
        print(f"[Init]  MapPoints: {len(self.mappoints)}")


        self.visualize_map_and_camera(self.poses[-1], desc=f" После фильтрации: точек: {len(self.mappoints)}", min_observations=2)

        return i, img_curr, kp_curr, des_curr
    
    def select_mappoint_candidates(
        self,
        pose: np.ndarray,
        image_shape: Tuple[int, int],
        min_observations: int = 2,
        depth_min: float = 1.0,
        depth_max: float = 40.0
    ):
        if pose is None:
            if len(self.poses) == 0:
                return []
            pose = self.poses[-1]

        h, w = image_shape
        candidates = []

        count_invalid = 0
        count_behind = 0
        count_outside = 0
        count_depth_filtered = 0
        count_low_observations = 0

        pose_inv = np.linalg.inv(pose)
        K = self.K

        print(f"[select_mappoint_candidates] Поза камеры (t): {pose[:3, 3]}")
        print(f"[select_mappoint_candidates] Всего MapPoints: {len(self.mappoints)}")

        for mp in self.mappoints:
            if mp.descriptor is None or not np.all(np.isfinite(mp.coord)):
                count_invalid += 1
                continue

            if len(mp.observations) < min_observations:
                count_low_observations += 1
                continue

            Pw_hom = np.append(mp.coord, 1.0)
            Pc = pose_inv @ Pw_hom

            z = Pc[2]
            if z <= 0:
                count_behind += 1
                continue

            if z < depth_min or z > depth_max:
                count_depth_filtered += 1
                continue

            u = K[0, 0] * (Pc[0] / z) + K[0, 2]
            v = K[1, 1] * (Pc[1] / z) + K[1, 2]

            if u < 0 or u >= w or v < 0 or v >= h:
                count_outside += 1
                continue

            candidates.append((mp, mp.descriptor))

        print(f"[select_mappoint_candidates] Пропущено:")
        print(f"  - невалидные (NaN/descriptor): {count_invalid}")
        print(f"  - мало наблюдений:             {count_low_observations}")
        print(f"  - за камерой (Z<0):            {count_behind}")
        print(f"  - вне допустимой глубины:      {count_depth_filtered}")
        print(f"  - вне изображения:             {count_outside}")
        print(f"[select_mappoint_candidates] Выбрано кандидатов: {len(candidates)}")

        return candidates

    def visualize_matches(self, delay_ms: int = 1000):
        num_pairs = len(self.keypoints)
        if num_pairs == 0:
            print("Нет сопоставлений. Сначала вызови run().")
            return

        print("Управление: [A] назад, [D] вперёд, [Q] выход")

        i = 0
        while True:
            kp1, kp2 = self.keypoints[i]
            matches = self.matches[i]
            img1 = self.frames[i]
            img2 = self.frames[i + 1]

            img1_color = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
            img2_color = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)

            vis = cv2.drawMatches(
                img1_color, kp1,
                img2_color, kp2,
                matches, None,
                matchColor=(0, 255, 0),
                singlePointColor=(255, 0, 0),
                flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
            )

            # Отрисовать текущий индекс
            text = f"Frame pair: {i} / {num_pairs - 1}"
            cv2.putText(vis, text, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 255), 2)

            cv2.imshow("Feature Matches (A/D/Q)", vis)
            key = cv2.waitKey(delay_ms) & 0xFF

            if key == ord('q'):
                print("Выход из визуализации.")
                break
            elif key == ord('d'):
                i = min(i + 1, num_pairs - 1)
            elif key == ord('a'):
                i = max(i - 1, 0)

        cv2.destroyAllWindows()

    def visualize_matches_once(
        self,
        img1, img2,
        kp1, kp2,
        matches,
        window_name: str = "Feature Matches (Q/Esc to close)",
        fraction: float = 1.0 
    ):
        if not matches:
            print("[VizOnce] Нет матчей для визуализации.")
            return

        # Копии изображений в цвете
        img1c = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR) if len(img1.shape) == 2 else img1.copy()
        img2c = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR) if len(img2.shape) == 2 else img2.copy()

        # Определяем сколько матчей показывать
        total_to_show = int(len(matches) * fraction)
        total_to_show = max(1, total_to_show)
        to_draw = matches[:total_to_show]

        # Цвета для матчей
        colors = [
            (0, 255, 0),   # зелёный
            (0, 0, 255),   # красный
            (255, 0, 0),   # синий
            (255, 255, 0), # голубой
            (255, 0, 255)  # фиолетовый
        ]

        # Отрисовка вручную
        vis = np.vstack((img1c, img2c))
        offset_y = img1c.shape[0]  # смещение по Y для второй картинки
        for i, m in enumerate(to_draw):
            pt1 = tuple(map(int, kp1[m.queryIdx].pt))
            pt2 = tuple(map(int, kp2[m.trainIdx].pt))
            pt2 = (pt2[0], pt2[1] + offset_y)  # сдвигаем по Y
            color = colors[i % len(colors)]
            cv2.circle(vis, pt1, 4, color, -1)
            cv2.circle(vis, pt2, 4, color, -1)
            cv2.line(vis, pt1, pt2, color, 1)


        # Надпись
        text = f"Matches: {len(matches)} | Shown: {len(to_draw)} ({fraction*100:.0f}%)"
        cv2.putText(vis, text, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 255), 2, cv2.LINE_AA)

        # Показ
        cv2.imshow(window_name, vis)
        while True:
            key = cv2.waitKey(30) & 0xFF
            if key in (ord('q'), 27):  # q или Esc
                break
            if cv2.getWindowProperty(window_name, cv2.WND_PROP_VISIBLE) < 1:
                break

        cv2.destroyWindow(window_name)

    def visualize_map_and_camera(self, pose=None, desc: Optional[str] = None, min_observations: int = 1):
        h, w = 720, 1280
        K = self.K

        points = []
        colors = []

        if len(self.mappoints) > 0:
            pose_inv = np.linalg.inv(pose) if pose is not None else np.eye(4)

            for mp in self.mappoints:
                if len(mp.observations) < min_observations:
                    continue
                if not np.all(np.isfinite(mp.coord)):
                    continue

                Pw_hom = np.append(mp.coord, 1.0)
                Pc = pose_inv @ Pw_hom

                if Pc[2] > 0:
                    u = K[0, 0] * (Pc[0] / Pc[2]) + K[0, 2]
                    v = K[1, 1] * (Pc[1] / Pc[2]) + K[1, 2]
                    if 0 <= u < w and 0 <= v < h:
                        color = [0, 1, 0]
                    else:
                        color = [1, 0, 0]
                else:
                    color = [1, 0, 0]

                points.append(mp.coord)
                colors.append(color)

        geometries = []

        if points:
            pcd = o3d.geometry.PointCloud()
            pcd.points = o3d.utility.Vector3dVector(np.array(points))
            pcd.colors = o3d.utility.Vector3dVector(np.array(colors))
            geometries.append(pcd)

        for T in self.poses:
            f = self.create_camera_frustum(scale=2.0)
            f.transform(T)
            geometries.append(f)

        if pose is not None:
            cam_center = o3d.geometry.TriangleMesh.create_sphere(radius=0.2)
            cam_center.translate(pose[:3, 3])
            cam_center.paint_uniform_color([1, 0.7, 0])
            geometries.append(cam_center)

        if not geometries:
            print("Нет данных для визуализации.")
            return

        # ----- Визуализация с установкой начального угла -----
        vis = o3d.visualization.Visualizer()
        vis.create_window(window_name=f"MapPoints and Cameras. {desc or ''}", width=1280, height=720)

        for g in geometries:
            vis.add_geometry(g)

        vis.poll_events()
        vis.update_renderer()

        # Настройка начального положения камеры
        ctr = vis.get_view_control()
        if pose is not None:
            ctr.set_lookat(pose[:3, 3])  # Центр обзора — текущая камера
        else:
            ctr.set_lookat([0, 0, 0])
            
        ctr.set_front([0.9, 0, 0])         # Взгляд сбоку под углом (X, Y, Z)
        ctr.set_up([0, 1, 0])               # Направление "вверх"
        ctr.set_zoom(0.6)                   # Масштаб (приближение)

        vis.run()
        vis.destroy_window()
      
    def show_mappoint_observations(self, mp_index: int, text: str = ""):
        if mp_index < 0 or mp_index >= len(self.mappoints):
            print(f"[Error] MapPoint index {mp_index} вне диапазона (0..{len(self.mappoints)-1})")
            return

        mp = self.mappoints[mp_index]
        if not mp.observations:
            print(f"[MapPoint #{mp_index}] Нет наблюдений.")
            return

        # print(f"[MapPoint #{mp_index}] Координаты 3D: {mp.coord}")
        # print(f"Всего наблюдений: {len(mp.observations)}")

        tiles = []

        for obs_id, (frame_id, kp_idx, pt2d) in enumerate(mp.observations):
            # print(f"  Observation {obs_id}: Frame #{frame_id}, KeyPoint #{kp_idx}, 2D: {pt2d}")

            if not (0 <= frame_id < len(self.frames)):
                continue

            img = self.frames[frame_id]
            imgc = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR) if len(img.shape) == 2 else img.copy()

            p = (int(round(pt2d[0])), int(round(pt2d[1])))
            cv2.circle(imgc, p, 5, (0, 0, 255), -1)
            cv2.circle(imgc, p, 9, (0, 0, 255), 2)
            header = f"MP#{mp_index} | Obs#{obs_id} | Frame {frame_id} | KP {kp_idx}"
            if text:
                header += f" | {text}"
            cv2.putText(imgc, header, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                        0.9, (0, 255, 255), 2, cv2.LINE_AA)

            tiles.append(imgc)

        if not tiles:
            print("[MapPoint] Нет валидных изображений для визуализации.")
            return

        target_w = max(t.shape[1] for t in tiles)
        resized = []
        for t in tiles:
            h, w = t.shape[:2]
            if w != target_w:
                scale = target_w / float(w)
                t = cv2.resize(t, (target_w, int(round(h * scale))), interpolation=cv2.INTER_AREA)
            resized.append(t)

        sep_h = 4
        sep = np.full((sep_h, target_w, 3), (40, 40, 40), dtype=np.uint8)
        mosaic_parts = []
        for i, t in enumerate(resized):
            mosaic_parts.append(t)
            if i != len(resized) - 1:
                mosaic_parts.append(sep)

        vis = np.vstack(mosaic_parts)

        win_name = f"MapPoint #{mp_index} — {len(tiles)} наблюдений"
        cv2.imshow(win_name, vis)
        while True:
            key = cv2.waitKey(30) & 0xFF
            if key in (ord('q'), 27):  # q или Esc
                break
            if cv2.getWindowProperty(win_name, cv2.WND_PROP_VISIBLE) < 1:
                break
        cv2.destroyWindow(win_name)
        
    def create_camera_frustum(self, scale: float = 0.05) -> o3d.geometry.LineSet:
        """
        Создаёт 3D-модель пирамиды камеры для визуализации в open3d.
        scale – масштаб фрустума
        """
        points = np.array([
            [0, 0, 0],
            [ scale,  scale,  scale * 2],
            [ scale, -scale,  scale * 2],
            [-scale, -scale,  scale * 2],
            [-scale,  scale,  scale * 2],
        ])
        lines = [
            [0, 1], [0, 2], [0, 3], [0, 4],
            [1, 2], [2, 3], [3, 4], [4, 1]
        ]
        frustum = o3d.geometry.LineSet()
        frustum.points = o3d.utility.Vector3dVector(points)
        frustum.lines = o3d.utility.Vector2iVector(lines)
        frustum.paint_uniform_color([0.0, 0.6, 1.0])
        return frustum
    
    def track_frame(self, img, kp, des):
        if len(self.mappoints) == 0 or des is None or len(kp) == 0:
            print("[TrackFrame] Нет MapPoints или дескрипторов.")
            return None

        pose_guess = self.poses[-1] if self.poses else np.eye(4)
        h, w = img.shape

        # 1. Выбор кандидатов по видимости
        candidates = self.select_mappoint_candidates(
            pose_guess, 
            img.shape, 
            min_observations=2, 
            depth_min=1, 
            depth_max=20.0
        )
        if not candidates:
            print("[TrackFrame] Нет кандидатов для трекинга.")
            return None

        des_mps = np.array([desc for _, desc in candidates])
        mp_list = [mp for mp, _ in candidates]

        # 2. Сопоставление дескрипторов
        knn_matches = self.matcher.knnMatch(des_mps, des, k=2)
        ratio_thresh = self.params.get("ratio_thresh", 0.6)

        good_matches = []
        matches_map = []

        for pair in knn_matches:
            if len(pair) < 2:
                continue
            m, n = pair
            if m.distance < ratio_thresh * n.distance:
                if m.queryIdx >= len(mp_list) or m.trainIdx >= len(kp):
                    print(f"[TrackFrame] Предупреждение: неверный индекс m.queryIdx={m.queryIdx}, m.trainIdx={m.trainIdx}")
                    continue
                mp = mp_list[m.queryIdx]
                matches_map.append((mp, m.trainIdx))
                good_matches.append(m)

        print(f"[TrackFrame] Совпадений после ratio test: {len(good_matches)}")

        if len(matches_map) < 4:
            print("[TrackFrame] Недостаточно валидных матчей.")
            return None

        object_points = np.array([mp.coord for mp, _ in matches_map])
        image_points = np.array([kp[kp_idx].pt for _, kp_idx in matches_map])

        if not np.isfinite(object_points).all() or not np.isfinite(image_points).all():
            print("[TrackFrame] Обнаружены NaN или Inf в точках.")
            return None

        depths = np.linalg.norm(object_points, axis=1)
        print(f"[TrackFrame] Глубины: min={depths.min():.2f}, max={depths.max():.2f}, mean={depths.mean():.2f}")
        
        print(f"[Debug] object_points.shape: {object_points.shape}")
        print(f"[Debug] image_points.shape: {image_points.shape}")
        print(f"[Debug] object_points[:5]:\n{object_points[:5]}")
        print(f"[Debug] image_points[:5]:\n{image_points[:5]}")
        print(f"[Debug] object_points Z std: {object_points[:,2].std():.5f}")
        print(f"[Debug] K:\n{self.K}")
        print(f"[Debug] Valid points: {np.isfinite(object_points).all()}, {np.isfinite(image_points).all()}")

        for m in good_matches:
            if m.trainIdx >= len(kp):
                print(f"[Warn] m.trainIdx={m.trainIdx} >= len(kp)={len(kp)}")

        success, rvec, tvec, inliers = cv2.solvePnPRansac(
            object_points,
            image_points,
            self.K,
            None,
            iterationsCount=2000,
            reprojectionError=50.0,
            confidence=0.99,
            flags=cv2.SOLVEPNP_ITERATIVE
        )

        if not success or inliers is None or len(inliers) < 6:
            print(f"[TrackFrame] PnP не удалось. Inliers: {0 if inliers is None else len(inliers)}")
            return None

        print(f"[TrackFrame] PnP успешен. Inliers: {len(inliers)} / {len(object_points)}")

        # 4. Reprojection error диагностика
        errors = []
        for i in inliers.flatten():
            pt3d = object_points[i].reshape(1, 3)
            pt2d = image_points[i].reshape(1, 2)
            proj, _ = cv2.projectPoints(pt3d, rvec, tvec, self.K, None)
            error = np.linalg.norm(proj.ravel() - pt2d.ravel())
            errors.append(error)

        if errors:
            print(f"[TrackFrame] Ошибка проекции (px): min={np.min(errors):.2f}, max={np.max(errors):.2f}, mean={np.mean(errors):.2f}")

        # 5. Формирование матрицы T
        R, _ = cv2.Rodrigues(rvec)
        T = np.eye(4)
        T[:3, :3] = R
        T[:3, 3] = tvec.ravel()

        print(f"[TrackFrame] Поза камеры: X={T[0,3]:.3f}, Y={T[1,3]:.3f}, Z={T[2,3]:.3f}")

        return T
    
    def match_mappoints_to_frame(self, des2, frame_id, kp2):
        if len(self.mappoints) == 0 or des2 is None:
            print(f"[MapPointMatch:{frame_id}] Нет MapPoints или дескрипторов для матчинга.")
            return 0

        des_mps = np.array([mp.descriptor for mp in self.mappoints])
        matches = self.matcher.match(des2, des_mps)
        print(f"[MapPointMatch:{frame_id}] Всего матчей найдено: {len(matches)}")

        new_obs = 0
        dublicates = 0
        for m in matches:
            mp = self.mappoints[m.trainIdx]
            already_seen = any(obs[0] == frame_id for obs in mp.observations)
            if already_seen:
                dublicates += 1
                continue
            pt = kp2[m.queryIdx].pt
            mp.observations.append((frame_id, m.queryIdx, pt))
            new_obs += 1

        print(f"[MapPointMatch:{frame_id}] Новых observation добавлено: {new_obs}")
        print(f"[MapPointMatch:{frame_id}] Дубликатов (уже были observation): {dublicates}")
        print(f"[MapPointMatch:{frame_id}] Всего MapPoints: {len(self.mappoints)}")
        print(f"[MapPointMatch:{frame_id}] Уникальных keypoints использовано: {len(set([m.queryIdx for m in matches]))}")

        return new_obs

    def find_mappoint_by_kp(self, kp_idx, search_frame_ids=None):
        """
        Ищет MapPoint, в котором есть наблюдение с данным kp_idx
        (в любом из заданных кадров или во всех)
        """
        for mp in self.mappoints:
            for obs in mp.observations:
                frame_id, obs_kp_idx, _ = obs
                if obs_kp_idx == kp_idx:
                    if search_frame_ids is None or frame_id in search_frame_ids:
                        return mp
        return None

    def find_mappoint_for_keypoint(self, frame_id: int, kp_idx: int):
        for mp in self.mappoints:
            for obs in mp.observations:
                if obs[0] == frame_id and obs[1] == kp_idx:
                    return mp
        return None
    
    def create_new_keyframe(self, frame_id, img, kp, des, pose, keyframes_size=3):
        print(f"[KeyFrame] ДОБАВЛЕН новый KeyFrame #{frame_id} (всего KeyFrames: {len(self.keyframes)+1})")
        self.add_keyframe(frame_id, img, kp, des, pose)

        total_new_points = 0

        for prev_kf in self.keyframes[-keyframes_size:]:
            if prev_kf.id == frame_id:
                continue
            kp1, kp2, matches, pts1, pts2 = self.get_matches(
                prev_kf.keypoints, prev_kf.descriptors, kp, des
            )

            if len(matches) < 8:
                continue

            X_world = self.triangulate_points(pts1, pts2, prev_kf.pose, pose)
            if X_world is None:
                continue

            added = 0
            for j, p in enumerate(X_world):
                kp_idx = matches[j].trainIdx

                if self.find_mappoint_for_keypoint(frame_id, kp_idx) is not None:
                    continue

                desc = des[kp_idx]
                self.add_mappoint(
                    coord_3d=p,
                    frame_id=frame_id,
                    kp_idx=kp_idx,
                    pt2d=tuple(pts2[j]),
                    descriptor=desc
                )
                added += 1

            total_new_points += added
            print(f"[KeyFrame]   -> Добавлено {added} новых MapPoints из пары с KeyFrame #{prev_kf.id}")
            
        print(f"[KeyFrame] Всего добавлено {total_new_points} новых MapPoints для KeyFrame #{frame_id}")

    def need_new_keyframe(self, T_curr: np.ndarray, keypoints: List[cv2.KeyPoint]) -> bool:
        # Параметры из self.params
        min_tracked = self.params.get("min_tracked_points",20)
        min_interval = self.params.get("keyframe_interval", 4)
        translation_thresh = self.params.get("keyframe_translation_thresh", 100)

        # Получаем индекс текущего кадра (последний добавленный в poses)
        current_frame_id = len(self.poses) - 1

        # Считаем, сколько MapPoints наблюдаются (замечены) на этом кадре:
        tracked_points = 0
        for mp in self.mappoints:
            for obs in mp.observations:
                if obs[0] == current_frame_id:
                    tracked_points += 1

        print(f"[KeyFrameDecision] Количество MapPoints, наблюдаемых в текущем кадре: {tracked_points}")


        # Проверяем минимальный интервал между KeyFrames
        frames_since_last_kf = len(self.poses) - 1 - self.keyframes[-1].id
        print(f"[KeyFrameDecision] Кадров с последнего KeyFrame: {frames_since_last_kf}")

        # Если трекается мало точек – добавляем KeyFrame
        if tracked_points < min_tracked:
            print("[KeyFrameDecision] Мало точек – добавляем KeyFrame.")
            return True

        # Проверяем смещение камеры
        last_kf_pose = self.keyframes[-1].pose
        delta = np.linalg.norm(T_curr[:3, 3] - last_kf_pose[:3, 3])
        print(f"[KeyFrameDecision] Смещение камеры: {delta:.3f}")

        if delta > translation_thresh:
            print("[KeyFrameDecision] Камера сильно сместилась – добавляем KeyFrame.")
            return True
        
        if frames_since_last_kf > min_interval:
            print("[KeyFrameDecision] Достаточно кадров прошло с последнего KeyFrame – добавляем новый.")
            return True

        print("[KeyFrameDecision] Новый KeyFrame не требуется.")
        return False

    def visualize_keypoints(self, img, kp, window_name="Keypoints"):
        img_vis = cv2.drawKeypoints(
            img, kp, None,
            color=(0, 255, 0),
            flags=cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS
        )
        cv2.imshow(window_name, img_vis)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
    def run(self):

        if len(self.frames) < 2:
            raise ValueError("Недостаточно кадров для запуска одометрии.")
        
        idx, img1, kp1, des1 = self.init_map()

        # # for i in range(idx + 1, len(self.frames)):
        # for i in range(idx + 1, 10):
        #     print(f"Обработка кадра {i} из {len(self.frames)}...")
        #     img2 = self.frames[i]
        #     kp2, des2 = self.detect_and_compute(img2)
            
        #     # self.visualize_keypoints(img2, kp2)

        #     if des2 is None or len(kp2) == 0:
        #         continue
            
        #     num_matched = self.match_mappoints_to_frame(des2, i, kp2)
        #     print(f"[Track] Сопоставлено MapPoints с кадром {i}: {num_matched}")

        #     T_curr = self.track_frame(img2, kp2, des2)
            
        #     if T_curr is None:
        #         raise RuntimeError(f"[Track] Не удалось оценить позу для кадра {i}.")
                  
        #     pose = self.poses[-1]
        #     print(f"[Track] Поза камеры для предыдущего кадра: {pose[:3, 3]}")
        #     # self.visualize_map_and_camera(pose, desc=f"Pose {i-1}")      
                            
        #     T_total = self.poses[-1] @ T_curr
        #     self.poses.append(T_total)   
            
        #     pose = self.poses[-1]
        #     print(f"[Track] Поза камеры для кадра {i}: {pose[:3, 3]}")
        #     # self.visualize_map_and_camera(pose, desc=f"Pose {i}")       

        #     if self.need_new_keyframe(T_curr, kp2):
        #         self.create_new_keyframe(i, img2, kp2, des2, T_curr, keyframes_size=3)
        #         self.bundle_adjustment_g2o(last_frame_size=5)

        #     img1, kp1, des1 = img2, kp2, des2

In [ ]:
import cv2
from pathlib import Path

VIDEO_PATH = DATA_PATH / 'KITTI_sequence_1/image_l'
CALIBRATION_PATH = DATA_PATH / 'KITTI_sequence_1/calib.txt'
TARGET_FPS = 2.0

if not VIDEO_PATH.exists():
    raise FileNotFoundError(f'Видео не найдено по пути: {VIDEO_PATH.resolve()}')

if not CALIBRATION_PATH.exists():
    raise FileNotFoundError(f'Калибровочный файл не найден: {CALIBRATION_PATH.resolve()}')

params = {
    "target_fps": TARGET_FPS,  # частота обработки кадров

    # ORB-параметры
    "nfeatures": 100,             # [default=500] больше фич — выше плотность покрытия сцены
    "fastThreshold": 10,          # [default=20] ниже порог — чувствительнее к слабым углам
    "edgeThreshold": 20,           # [default=31] позволяет детектировать ближе к краям кадра
    "scaleFactor": 1.2,           # [default=1.2] масштаб между уровнями пирамиды (меньше — медленнее, но точнее)
    "nlevels": 8,                 # [default=8] число уровней пирамиды — выше = лучше устойчивость к масштабу

    # Фильтрация матчей по дистанции (Lowe’s ratio test)
    "ratio_thresh": 0.75,          # [default=0.75] меньше строгая фильтрация
    "max_match_distance": 200,    # максимальная допустимая дистанция матча
    # "use_mutual_check": True,      # включить/выключить взаимную проверку

    # Grid-детектор (разделение кадра на сетку)
    "use_grid": False,             # включение/выключение сетки
    "grid_rows": 4,
    "grid_cols": 6,

    # ANMS (равномерное распределение фичей)
    "use_anms": True,             # включить/выключить ANMS после ORB
    "anms_count": 3000,            # сколько фич оставить после ANMS (подбери под свою сцену)
    "anms_tolerance": 0.2,  # порог ANMS (чем меньше, тем более равномерно распределены фичи)

    # FLANN-матчер (быстрый поиск похожих фичей)
    "flann_table_number": 10,
    "flann_key_size": 14,
    "flann_multi_probe_level": 2,
    "flann_checks": 50,
    
    "init_keyframes": 10,         # минимальное число KeyFrame для инициализации карты
    "last_frame_size": 10,         # количество последних кадров для g2o Bundle Adjustment
    "g2o_optimize_step": 20,  # шаг оптимизации g2o (количество итераций)
    "ba_obs_size": 2,            # минимальное число наблюдений для Bundle Adjustment

    # Добавь сюда другие нужные параметры (например, для keyframe/track decision)
    "min_tracked_points": 30,      # минимальное число трекаемых MapPoints для удержания трекинга
    "keyframe_interval": 5,        # минимальное число кадров между KeyFrame
    # "keyframe_translation_thresh": 0.2, # минимальное смещение камеры для нового KeyFrame
}

vo = VisualOdometry(
    video_path=VIDEO_PATH,
    calibration_file=CALIBRATION_PATH,
    **params
)

[KITTI] Загружено 51 кадров из image_l


In [130]:
# from IPython.display import display, clear_output
# from PIL import Image
# import time
# import cv2

# num_frames_to_show = int(TARGET_FPS * 30)
# frames_to_show = vo.frames[:num_frames_to_show]

# for frame in frames_to_show:
#     rgb_frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)
#     clear_output(wait=True)
#     display(Image.fromarray(rgb_frame))
#     time.sleep(1 / TARGET_FPS / 2)

In [131]:
vo.clear_data()
vo.run()

[Init] Запуск инициализации карты...
[Debug] Найдено 8105 ключевых точек
[Debug] После ANMS осталось 2763 точек
[Init] Первый кадр: #0, найдено 2763 ключевых точек.
[Init:1] Обработка кадра #1..............
[Debug] Найдено 8196 ключевых точек
[Debug] После ANMS осталось 2860 точек
[Init:1] Сопоставлено 676 фич между кадрами 0 и 1
[Init:1] Средний параллакс: 15.42
[Pose:1] Камера на позиции: X=0.013, Y=-0.021, Z=1.000
[Init:1] Добавлен KeyFrame #1
[Triangulate] Всего: 676 | валидных Z∈(0.01,200.0): 642
[Init:1] Триангуляция дала 642 3D-точек.
[Init:1] Добавлено 642 новых 3D-точек после фильрации.
[Init:1] Добавлено 0 новых наблюдений к существующим MapPoints.
[Init:1] KeyFrames: 2 | MapPoints: 642
[Init:2] Обработка кадра #2..............
[Debug] Найдено 8286 ключевых точек
[Debug] После ANMS осталось 2850 точек
[Init:2] Сопоставлено 562 фич между кадрами 1 и 2
[Init:2] Средний параллакс: 18.34
[Pose:2] Камера на позиции: X=0.009, Y=-0.054, Z=1.999
[Init:2] Добавлен KeyFrame #2
[Triangu

In [132]:
# vo.visualize_matches()

In [133]:
import open3d as o3d
import numpy as np

def create_camera_frustum(scale: float = 0.05) -> o3d.geometry.LineSet:
    """
    Создаёт 3D-модель пирамиды камеры для визуализации в open3d.
    scale – масштаб фрустума
    """
    points = np.array([
        [0, 0, 0],
        [ scale,  scale,  scale * 2],
        [ scale, -scale,  scale * 2],
        [-scale, -scale,  scale * 2],
        [-scale,  scale,  scale * 2],
    ])
    lines = [
        [0, 1], [0, 2], [0, 3], [0, 4],
        [1, 2], [2, 3], [3, 4], [4, 1]
    ]
    frustum = o3d.geometry.LineSet()
    frustum.points = o3d.utility.Vector3dVector(points)
    frustum.lines = o3d.utility.Vector2iVector(lines)
    frustum.paint_uniform_color([0.0, 0.6, 1.0])
    return frustum


def check_and_visualize_mappoints(vo):
    """
    Проверяет и визуализирует 3D MapPoints после инициализации.
    vo: экземпляр класса VisualOdometry
    """
    if not vo.mappoints:
        print("[Check] Нет MapPoints для отображения.")
        return

    coords = np.array([mp.coord for mp in vo.mappoints if np.all(np.isfinite(mp.coord))])

    if coords.size == 0:
        print("[Check] Все точки невалидны.")
        return

    # Статистика
    depths = coords[:, 2]
    print(f"[Check] Всего точек: {len(coords)}")
    print(f"[Check] Глубина Z: mean={depths.mean():.3f}, median={np.median(depths):.3f}, min={depths.min():.3f}, max={depths.max():.3f}")

    print(f"[Check] X: mean={coords[:,0].mean():.3f}, min={coords[:,0].min():.3f}, max={coords[:,0].max():.3f}")
    print(f"[Check] Y: mean={coords[:,1].mean():.3f}, min={coords[:,1].min():.3f}, max={coords[:,1].max():.3f}")

    print("[Check] Примеры точек:")
    for i in range(min(5, len(coords))):
        print(f"   {coords[i]}")

    # Создаём PointCloud для Open3D
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(coords)
    pcd.paint_uniform_color([1.0, 0.0, 0.0])  # красные точки

    # Добавляем камеры (если есть позы)
    geometries = [pcd]
    for pose in vo.poses:
        frustum = create_camera_frustum(scale=5).transform(pose)
        geometries.append(frustum)

    # Визуализация
    o3d.visualization.draw_geometries(geometries, window_name="MapPoints после инициализации")

# check_and_visualize_mappoints(vo)